In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('rice disease detection')

In [ ]:
# ===========================
# 1. IMPORT LIBRARIES
# ===========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# ===========================
# 2. OPTIMIZED PARAMETERS
# ===========================
IMG_SIZE = 224  # Standard size for MobileNetV2
BATCH_SIZE = 32  # Proper batch size
EPOCHS = 50
LEARNING_RATE = 0.001  # Standard starting LR
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

# ===========================
# 3. DATASET PATHS
# ===========================
BASE_DIR = '/kaggle/input/rice-dataset'
train_dir = os.path.join(BASE_DIR, 'Rice_Leaf_Disease/Rice_Leaf_Diease/Rice_Leaf_Diease/train')
test_dir = os.path.join(BASE_DIR, 'Rice_Leaf_Disease/Rice_Leaf_Diease/Rice_Leaf_Diease/test')

print("="*70)
print("DATASET SETUP")
print("="*70)
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")

# ===========================
# 4. GET CLASS NAMES DIRECTLY FROM FOLDERS
# ===========================
# Don't hardcode - let Keras auto-detect!
train_classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
test_classes = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])

print(f"\nTrain classes ({len(train_classes)}):")
for cls in train_classes:
    count = len(os.listdir(os.path.join(train_dir, cls)))
    print(f"  {cls}: {count} images")

print(f"\nTest classes ({len(test_classes)}):")
for cls in test_classes:
    count = len(os.listdir(os.path.join(test_dir, cls)))
    print(f"  {cls}: {count} images")

# Create mapping: normalize test names to match train names
def normalize_name(name):
    return name.lower().replace(' ', '_')

# Build train class mapping (normalized -> actual)
train_norm_map = {normalize_name(c): c for c in train_classes}
test_norm_map = {normalize_name(c): c for c in test_classes}

# Get consistent normalized class list (from train)
CLASSES_NORMALIZED = sorted(train_norm_map.keys())
NUM_CLASSES = len(CLASSES_NORMALIZED)

# Map to actual folder names for generators
TRAIN_CLASSES_ACTUAL = [train_norm_map[c] for c in CLASSES_NORMALIZED]
TEST_CLASSES_ACTUAL = [test_norm_map.get(c, c) for c in CLASSES_NORMALIZED]

print(f"\n✅ Using {NUM_CLASSES} classes in normalized order:")
for i, (norm, train_actual, test_actual) in enumerate(zip(CLASSES_NORMALIZED, TRAIN_CLASSES_ACTUAL, TEST_CLASSES_ACTUAL)):
    print(f"  {i}: {norm:30s} | Train: {train_actual:30s} | Test: {test_actual:30s}")

# ===========================
# 5. DATA GENERATORS WITH CORRECT PREPROCESSING
# ===========================
print("\n" + "="*70)
print("CREATING DATA GENERATORS")
print("="*70)

# CRITICAL: Use preprocessing_function for MobileNetV2
# This applies the correct normalization (-1 to 1 range)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # MobileNetV2 preprocessing!
    rotation_range=20,  # Moderate augmentation
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Same preprocessing!
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=TRAIN_CLASSES_ACTUAL,
    subset='training',
    shuffle=True,
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=TRAIN_CLASSES_ACTUAL,
    subset='validation',
    shuffle=False,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=TEST_CLASSES_ACTUAL,
    shuffle=False
)

print(f"\n✅ Train samples: {train_generator.samples}")
print(f"✅ Validation samples: {validation_generator.samples}")
print(f"✅ Test samples: {test_generator.samples}")
print(f"✅ Steps per epoch: {len(train_generator)}")

# Verify class indices
print(f"\n📊 Class indices verification:")
print(f"Train: {train_generator.class_indices}")
print(f"Test: {test_generator.class_indices}")

# ===========================
# 6. BUILD MODEL - MobileNetV2 (Lightweight & Fast)
# ===========================
print("\n" + "="*70)
print("BUILDING MODEL - MobileNetV2")
print("="*70)

def build_mobilenet_model(num_classes, img_size=224):
    """Build MobileNetV2-based model"""
    
    inputs = layers.Input(shape=(img_size, img_size, 3))
    
    # Load MobileNetV2 (much lighter than EfficientNetB3)
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs,
        pooling='avg'
    )
    
    # Start with frozen base
    base_model.trainable = False
    
    # Simple classification head
    x = base_model.output
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model, base_model

model, base_model = build_mobilenet_model(NUM_CLASSES, IMG_SIZE)

print(f"✅ Total parameters: {model.count_params():,}")
print(f"✅ Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

# ===========================
# 7. COMPILE MODEL
# ===========================
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ===========================
# 8. CALLBACKS
# ===========================
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        mode='max',
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_rice_mobilenet.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

# ===========================
# 9. TRAIN - PHASE 1 (Frozen Base)
# ===========================
print("\n" + "="*70)
print("PHASE 1: Training with Frozen Base (15 epochs)")
print("="*70)

history1 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

print(f"\n📊 Phase 1 Best Val Accuracy: {max(history1.history['val_accuracy']):.4f}")

# ===========================
# 10. TRAIN - PHASE 2 (Fine-tuning)
# ===========================
print("\n" + "="*70)
print("PHASE 2: Fine-tuning (Unfreeze last layers)")
print("="*70)

# Unfreeze last 30 layers
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

print(f"Trainable layers: {sum([layer.trainable for layer in model.layers])}")

# Recompile with lower LR
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE/10),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    initial_epoch=len(history1.history['loss']),
    callbacks=callbacks,
    verbose=1
)

# ===========================
# 11. COMBINE & PLOT HISTORY
# ===========================
history_combined = {
    'accuracy': history1.history['accuracy'] + history2.history['accuracy'],
    'val_accuracy': history1.history['val_accuracy'] + history2.history['val_accuracy'],
    'loss': history1.history['loss'] + history2.history['loss'],
    'val_loss': history1.history['val_loss'] + history2.history['val_loss']
}

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history_combined['accuracy'], label='Train', linewidth=2)
axes[0].plot(history_combined['val_accuracy'], label='Validation', linewidth=2)
axes[0].axvline(x=15, color='red', linestyle='--', label='Fine-tuning starts')
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Loss
axes[1].plot(history_combined['loss'], label='Train', linewidth=2)
axes[1].plot(history_combined['val_loss'], label='Validation', linewidth=2)
axes[1].axvline(x=15, color='red', linestyle='--', label='Fine-tuning starts')
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

best_val_acc = max(history_combined['val_accuracy'])
best_epoch = history_combined['val_accuracy'].index(best_val_acc) + 1
print(f"\n📊 Best Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%) at epoch {best_epoch}")

# ===========================
# 12. LOAD BEST MODEL & EVALUATE
# ===========================
print("\n" + "="*70)
print("LOADING BEST MODEL & EVALUATING ON TEST SET")
print("="*70)

model = keras.models.load_model('best_rice_mobilenet.h5')

test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print(f"\n{'='*70}")
print("TEST RESULTS")
print(f"{'='*70}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# ===========================
# 13. PREDICTIONS & CLASSIFICATION REPORT
# ===========================
test_generator.reset()
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_true, y_pred, target_names=CLASSES_NORMALIZED, digits=4))

# ===========================
# 14. CONFUSION MATRICES
# ===========================
cm = confusion_matrix(y_true, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(22, 9))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASSES_NORMALIZED,
            yticklabels=CLASSES_NORMALIZED,
            ax=axes[0])
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Normalized
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Greens',
            xticklabels=CLASSES_NORMALIZED,
            yticklabels=CLASSES_NORMALIZED,
            ax=axes[1])
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

# ===========================
# 15. VISUALIZE PREDICTIONS
# ===========================
def visualize_predictions(n=16):
    test_generator.reset()
    images_raw, labels = next(test_generator)
    
    # Reverse preprocessing to display properly
    images_display = (images_raw + 1.0) / 2.0  # MobileNetV2 uses [-1, 1] range
    
    preds = model.predict(images_raw[:n], verbose=0)
    
    fig, axes = plt.subplots(4, 4, figsize=(16, 16))
    axes = axes.flatten()
    
    for i in range(n):
        axes[i].imshow(images_display[i])
        
        true_idx = np.argmax(labels[i])
        pred_idx = np.argmax(preds[i])
        true_label = CLASSES_NORMALIZED[true_idx]
        pred_label = CLASSES_NORMALIZED[pred_idx]
        conf = preds[i][pred_idx] * 100
        
        color = 'green' if true_idx == pred_idx else 'red'
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\n{conf:.1f}%',
                         fontsize=9, color=color, fontweight='bold')
        axes[i].axis('off')
    
    plt.suptitle('Model Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_predictions(16)

# ===========================
# 16. PER-CLASS METRICS
# ===========================
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

df = pd.DataFrame({
    'Class': CLASSES_NORMALIZED,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\n" + "="*70)
print("PER-CLASS PERFORMANCE")
print("="*70)
print(df.to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].barh(CLASSES_NORMALIZED, precision, color='steelblue')
axes[0].set_xlabel('Precision')
axes[0].set_title('Precision by Class', fontweight='bold')
axes[0].set_xlim([0, 1])

axes[1].barh(CLASSES_NORMALIZED, recall, color='coral')
axes[1].set_xlabel('Recall')
axes[1].set_title('Recall by Class', fontweight='bold')
axes[1].set_xlim([0, 1])

axes[2].barh(CLASSES_NORMALIZED, f1, color='mediumseagreen')
axes[2].set_xlabel('F1-Score')
axes[2].set_title('F1-Score by Class', fontweight='bold')
axes[2].set_xlim([0, 1])

plt.tight_layout()
plt.show()

# ===========================
# 17. SAVE MODEL
# ===========================
model.save('rice_disease_mobilenet_final.h5')

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print(f"✅ Best model: best_rice_mobilenet.h5")
print(f"✅ Final model: rice_disease_mobilenet_final.h5")
print(f"✅ Test Accuracy: {test_accuracy*100:.2f}%")
print("="*70)

# ===========================
# 18. PREDICTION FUNCTION
# ===========================
def predict_image(image_path):
    """Predict disease for a single image"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    
    # Apply same preprocessing as training
    img_preprocessed = preprocess_input(img_resized)
    img_batch = np.expand_dims(img_preprocessed, axis=0)
    
    # Predict
    pred = model.predict(img_batch, verbose=0)[0]
    pred_idx = np.argmax(pred)
    pred_class = CLASSES_NORMALIZED[pred_idx]
    confidence = pred[pred_idx] * 100
    
    # Get top 3
    top3_idx = np.argsort(pred)[-3:][::-1]
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(img)
    axes[0].set_title('Input Image', fontweight='bold')
    axes[0].axis('off')
    
    top3_classes = [CLASSES_NORMALIZED[i] for i in top3_idx]
    top3_probs = [pred[i]*100 for i in top3_idx]
    axes[1].barh(top3_classes, top3_probs, color=['green', 'steelblue', 'coral'])
    axes[1].set_xlabel('Confidence (%)')
    axes[1].set_title('Top 3 Predictions', fontweight='bold')
    axes[1].set_xlim([0, 100])
    
    plt.suptitle(f'Prediction: {pred_class} ({confidence:.2f}%)', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return pred_class, confidence

print("\n✅ Prediction function ready!")
print("Usage: predict_image('/path/to/image.jpg')")

2026-01-12 11:43:19.080460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768218199.511091      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768218199.640439      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768218200.612190      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768218200.612231      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768218200.612234      55 computation_placer.cc:177] computation placer alr

TensorFlow Version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
DATASET SETUP
Train: /kaggle/input/rice-dataset/Rice_Leaf_Disease/Rice_Leaf_Diease/Rice_Leaf_Diease/train
Test: /kaggle/input/rice-dataset/Rice_Leaf_Disease/Rice_Leaf_Diease/Rice_Leaf_Diease/test

Train classes (10):
  bacterial_leaf_blight: 1386 images
  brown_spot: 1480 images
  healthy: 1491 images
  leaf_blast: 1801 images
  leaf_scald: 1670 images
  narrow_brown_spot: 1416 images
  neck_blast: 1000 images
  rice_hispa: 1461 images
  sheath_blight: 1578 images
  tungro: 1740 images

Test classes (10):
  Neck_Blast: 322 images
  Rice Hispa: 225 images
  Sheath Blight: 288 images
  Tungro: 310 images
  bacterial_leaf_blight: 376 images
  brown_spot: 380 images
  healthy: 391 images
  leaf_blast: 362 images
  leaf_scald: 386 images
  narrow_brown_spot: 382 images

✅ Using 10 classes in normalized order:
  0: bact

I0000 00:00:1768218230.787510      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1768218230.791399      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Total parameters: 2,423,242
✅ Trainable parameters: 165,258

PHASE 1: Training with Frozen Base (15 epochs)
Epoch 1/15


I0000 00:00:1768218241.693909     153 service.cc:152] XLA service 0x79b8881437a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768218241.693955     153 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1768218241.693960     153 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1768218243.058819     153 cuda_dnn.cc:529] Loaded cuDNN version 91002
2026-01-12 11:44:12.352045: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-12 11:44:12.488782: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
I0000 00:00:1768218254.989288     153 device_co

...

376/376 ━━━━━━━━━━━━━━━━━━━━ 322s 856ms/step - accuracy: 0.9779 - loss: 0.0613 - val_accuracy: 0.9814 - val_loss: 0.0508 - learning_rate: 1.0000e-04
Epoch 31/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 683ms/step - accuracy: 0.9794 - loss: 0.0601
Epoch 31: val_accuracy improved from 0.98135 to 0.98335, saving model to best_rice_mobilenet.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 322s 857ms/step - accuracy: 0.9794 - loss: 0.0601 - val_accuracy: 0.9833 - val_loss: 0.0572 - learning_rate: 1.0000e-04
Epoch 32/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 681ms/step - accuracy: 0.9816 - loss: 0.0546
Epoch 32: val_accuracy did not improve from 0.98335
376/376 ━━━━━━━━━━━━━━━━━━━━ 318s 845ms/step - accuracy: 0.9816 - loss: 0.0546 - val_accuracy: 0.9817 - val_loss: 0.0485 - learning_rate: 1.0000e-04
Epoch 33/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - accuracy: 0.9857 - loss: 0.0454
Epoch 33: val_accuracy improved from 0.98335 to 0.98435, saving model to best_rice_mobilenet.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 316s 841ms/step - accuracy: 0.9857 - loss: 0.0454 - val_accuracy: 0.9843 - val_loss: 0.0431 - learning_rate: 1.0000e-04
Epoch 34/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 676ms/step - accuracy: 0.9839 - loss: 0.0443
Epoch 34: val_accuracy improved from 0.98435 to 0.98468, saving model to best_rice_mobilenet.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 319s 849ms/step - accuracy: 0.9839 - loss: 0.0443 - val_accuracy: 0.9847 - val_loss: 0.0407 - learning_rate: 1.0000e-04
Epoch 35/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 666ms/step - accuracy: 0.9793 - loss: 0.0551
Epoch 35: val_accuracy improved from 0.98468 to 0.99068, saving model to best_rice_mobilenet.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 313s 832ms/step - accuracy: 0.9793 - loss: 0.0551 - val_accuracy: 0.9907 - val_loss: 0.0245 - learning_rate: 1.0000e-04
Epoch 36/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 679ms/step - accuracy: 0.9840 - loss: 0.0454
Epoch 36: val_accuracy did not improve from 0.99068
376/376 ━━━━━━━━━━━━━━━━━━━━ 319s 849ms/step - accuracy: 0.9840 - loss: 0.0454 - val_accuracy: 0.9830 - val_loss: 0.0514 - learning_rate: 1.0000e-04
Epoch 37/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.9837 - loss: 0.0451
Epoch 37: val_accuracy did not improve from 0.99068
376/376 ━━━━━━━━━━━━━━━━━━━━ 323s 859ms/step - accuracy: 0.9837 - loss: 0.0451 - val_accuracy: 0.9820 - val_loss: 0.0472 - learning_rate: 1.0000e-04
Epoch 38/50
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 693ms/step - accuracy: 0.9847 - loss: 0.0450
Epoch 38: val_accuracy did not improve from 0.99068
376/376 ━━━━━━━━━━━━━━━━━━━━ 329s 876ms/step - accuracy: 0.9847 - loss: 0.0450 - val_accuracy: 0.9843 - val_loss: 0.0352 - learnin